# 4. Metaprogramming

Julia has extensive support of *metaprogramming*, that is, writing programs that generate other programs.

Julia expressions are actually Julia objects:

In [2]:
ex = :(sin(x)+2) # : quotes the expression; can also use "quote" blocks

:(sin(x) + 2)

In [3]:
typeof(ex)

Expr

In [5]:
ex.args


3-element Array{Any,1}:
  :+       
  :(sin(x))
 2         

*Macros* are functions that are applied to expressions that in turn generate other expressions. This is then evaluated

In [6]:
macro mytime(ex)
    quote
        start = time()
        result = $ex
        stop = time()
        println("time: $(stop-start)")
        result
    end
end

In [7]:
@mytime randn(100,100)*randn(100,100)

time: 0.3491530418395996


100x100 Array{Float64,2}:
  -2.71914     6.6999     -1.30577  …    5.2316    -11.0382       0.90014 
  -7.13392     2.20583   -13.3039        5.1261      4.99885     -2.28197 
  -0.854808   -8.14914    15.6467       -6.13293    -0.0494801   -8.84625 
  12.7186     -2.26214     6.33962      -2.63817    -4.99795    -26.5111  
  -3.24379     7.64903   -12.3051        4.2369     -0.777312   -11.3241  
   9.27228    10.9853     -8.13451  …   10.9392     -7.1425       5.38652 
 -17.8695     -8.26481     5.51744       0.62028     4.14776     10.1411  
   1.5181     12.7657     -4.53762       1.21105     2.57646      1.47551 
   8.68605    23.069       1.41747       1.34449     7.15347      9.00923 
  -3.3913    -27.2045     13.3734        3.84731    -0.104065    -9.45328 
  20.9632    -11.4956      3.9695   …   -1.85235     4.48386      5.15321 
 -12.8498     -4.78087    -3.06139      -4.22939   -14.1557       6.70511 
  -3.731       1.02499     6.70089      -0.454728   -0.815443    12.801   

Julia actually provides its own better `@time` macro:

In [8]:
@time randn(100,100)*randn(100,100)

100x100 Array{Float64,2}:
  -2.64663     3.14769     11.8085    …    6.68086    22.4838     2.18353 
   5.13249     4.85234    -15.2032         3.86468    17.6687     1.12693 
  10.8434    -11.4068       5.09745       -9.57045     8.22124   -3.18819 
   2.90809     1.43358    -18.782          7.63109    -2.12416    4.93513 
   0.928754   -5.21001     13.2987        -1.27955    -5.9208    -5.98532 
 -15.094       0.0747849   -6.40863   …    6.16544   -25.1429     1.4259  
   4.32416     3.70796     23.7682       -17.6056      5.96162   -0.784301
   6.22989    -0.193696    -1.29534        3.9813     -5.20838   -1.15591 
  -0.650502    7.75045    -19.7026        -6.71136    -1.04207    5.64205 
  -9.20611    -5.52086      6.75775      -16.3488    -14.5403   -16.6278  
   8.46163     3.46066    -23.7672    …    9.8057     -6.89556   -9.4024  
   4.34024    -7.75575    -26.1231        28.0131    -11.3154    10.9155  
  -6.78092     7.07685    -10.6505       -12.0038    -10.5035     4.81162 

0.000206 seconds (155 allocations: 244.745 KB)


Metaprogramming is extremely powerful, but a bit of a dark art (for example, you have to be careful of *hygiene*, that is, not modifying symbols defined elsewhere).

I find the `macroexpand` function useful to figure out what is going on:

In [9]:
macroexpand(:(@mytime sin(x)))

quote  # In[6], line 3:
    #56#start = time() # In[6], line 4:
    #57#result = sin(x) # In[6], line 5:
    #58#stop = time() # In[6], line 6:
    println("time: $(#58#stop - #56#start)") # In[6], line 7:
    #57#result
end

> **Exercise**: write a macro that prints the values of the inputs to a function, and returns the result e.g.
> 
> `@printinput sin(x)`
> 
> should print the value of `x`, then return the result of `sin(x)`.

Some particularly useful macros included in Julia:
* `@time`, `@elapsed` for evaluating performance
* `@printf` for formatted printing
* `@code_warntype` for finding type unstable functions
* `@label`/`@goto` for when you wish you were using Fortran (or just want to translate code).

In [11]:
@printf "%.60f" 0.1

0.100000000000000005551115123125782702118158340454101562500000

In [16]:
function harmonicmean1(X::Array)
    s = 0
    for x in X
        s += 1/x
    end
    1/s
end

harmonicmean1 (generic function with 2 methods)

In [17]:
X = rand(1_000_000)
@code_warntype harmonicmean1(X)

Variables:
  X::Array{Float64,1}
  s::ANY
  #s35::Int64
  x::Float64

Body:
  begin  # In[16], line 2:
      s = 0 # In[16], line 3:
      #s35 = 1
      GenSym(2) = (Base.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s35::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(2),1))::Bool)) goto 1
      2: 
      GenSym(4) = (Base.arrayref)(X::Array{Float64,1},#s35::Int64)::Float64
      GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s35::Int64,1))
      x = GenSym(4)
      #s35 = GenSym(5) # In[16], line 4:
      s = s::UNION{FLOAT64,INT64} + (Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,1)),x::Float64))::Float64
      3: 
      GenSym(3) = (Base.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)((Base.box)(Base.Bool,(Base.not_int)(#s35::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(3),1))::Bool)))) goto 2
      1: 
      0:  # In[16], line 6:
      return 1 / s::UNIO

In [18]:
@time harmonicmean1(X)
@time harmonicmean1(X)

6.493513520815289e-8

0.051536 seconds (2.00 M allocations: 30.529 MB, 9.13% gc time)
  0.033154 seconds (2.00 M allocations: 30.518 MB, 15.08% gc time)


In [19]:
function harmonicmean2(X)
    s = 0.0
    for x in X
        s += 1/x
    end
    1/s
end

harmonicmean2 (generic function with 1 method)

In [20]:
@code_warntype harmonicmean2(X)

Variables:
  X::Array{Float64,1}
  s::Float64
  #s34::Int64
  x::Float64

Body:
  begin  # In[19], line 2:
      s = 0.0 # In[19], line 3:
      #s34 = 1
      GenSym(2) = (Base.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s34::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(2),1))::Bool)) goto 1
      2: 
      GenSym(4) = (Base.arrayref)(X::Array{Float64,1},#s34::Int64)::Float64
      GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s34::Int64,1))
      x = GenSym(4)
      #s34 = GenSym(5) # In[19], line 4:
      s = (Base.box)(Base.Float64,(Base.add_float)(s::Float64,(Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,1)),x::Float64))))
      3: 
      GenSym(3) = (Base.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)((Base.box)(Base.Bool,(Base.not_int)(#s34::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(3),1))::Bool)))) goto 2
      1: 
      0:  # In[19], line 6:

In [22]:
@time harmonicmean2(X)
@time harmonicmean2(X)

6.493513520815289e-8

0.001622 seconds (5 allocations: 176 bytes)
  0.001585 seconds (5 allocations: 176 bytes)


## What happens under the hood

1. Parsing: syntax is parsed (see `parse` function)
 - Macros are expanded
2. Code lowering: this is a simplified syntax (see `@code_lowered`)
 - `a += a` is changed to `a = a + b`
 - loops are replaced with gotos
3. Type inference (see `@code_typed`)
 - tries to infer the type of each object
 - decides which method should be dispatched
4. LLVM code generation (`@code_llvm`)
 - generates LLVM IR (intermediate representation) code
 - some initial optimisation passes 
5. Machine code (`@code_native`)
 - machine-specific optimisations
 - assembly code generation





In [23]:
@code_lowered harmonicmean2(X)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:X], Any[Any[Any[:X,:Any,0],Any[:s,:Any,2],Any[symbol("#s34"),:Any,2],Any[:x,:Any,18]],Any[],2,Any[]], :(begin  # In[19], line 2:
        s = 0.0 # In[19], line 3:
        GenSym(0) = X
        #s34 = (top(start))(GenSym(0))
        unless (top(!))((top(done))(GenSym(0),#s34)) goto 1
        2: 
        GenSym(1) = (top(next))(GenSym(0),#s34)
        x = (top(getfield))(GenSym(1),1)
        #s34 = (top(getfield))(GenSym(1),2) # In[19], line 4:
        s = s + 1 / x
        3: 
        unless (top(!))((top(!))((top(done))(GenSym(0),#s34))) goto 2
        1: 
        0:  # In[19], line 6:
        return 1 / s
    end))))

In [24]:
@code_typed harmonicmean2(X)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:X], Any[Any[Any[:X,Array{Float64,1},0],Any[:s,Float64,2],Any[symbol("#s34"),Int64,2],Any[:x,Float64,18]],Any[],Any[Array{Float64,1},Tuple{Float64,Int64},Int64,Int64,Float64,Int64],Any[]], :(begin  # In[19], line 2:
        s = 0.0 # In[19], line 3:
        #s34 = 1
        GenSym(2) = (Base.arraylen)(X::Array{Float64,1})::Int64
        unless (Base.box)(Base.Bool,(Base.not_int)(#s34::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(2),1))::Bool)) goto 1
        2: 
        GenSym(4) = (Base.arrayref)(X::Array{Float64,1},#s34::Int64)::Float64
        GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s34::Int64,1))
        x = GenSym(4)
        #s34 = GenSym(5) # In[19], line 4:
        s = (Base.box)(Base.Float64,(Base.add_float)(s::Float64,(Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,1)),x::Float64))))
        3: 
        GenSym(3) = (Base.arraylen)(X::Array{Float64,1})::Int64
        unless (Base.box

In [25]:
@code_llvm harmonicmean2(X)


define double @julia_harmonicmean2_22011(%jl_value_t*) {
top:
  %1 = getelementptr inbounds %jl_value_t* %0, i64 1
  %2 = bitcast %jl_value_t* %1 to i64*
  %3 = load i64* %2, align 8
  %4 = icmp eq i64 %3, 0
  br i1 %4, label %L3, label %L.preheader

L.preheader:                                      ; preds = %top
  %5 = load i64* %2, align 8
  %6 = bitcast %jl_value_t* %0 to i8**
  br label %L

L:                                                ; preds = %idxend, %L.preheader
  %"#s34.0" = phi i64 [ %14, %idxend ], [ 1, %L.preheader ]
  %s.0 = phi double [ %16, %idxend ], [ 0.000000e+00, %L.preheader ]
  %7 = add i64 %"#s34.0", -1
  %8 = icmp ult i64 %7, %5
  br i1 %8, label %idxend, label %oob

oob:                                              ; preds = %L
  %9 = alloca i64, align 8
  store i64 %"#s34.0", i64* %9, align 8
  call void @jl_bounds_error_ints(%jl_value_t* %0, i64* %9, i64 1)
  unreachable

idxend:                                           ; preds = %L
  %10 = load i8** %

In [26]:
@code_native harmonicmean2(X)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[19]
Source line: 3
	pushq	%rbp
	movq	%rsp, %rbp
	xorps	%xmm1, %xmm1
Source line: 3
	cmpq	$0, 8(%rdi)
	je	L98
	movq	8(%rdi), %r8
	movq	%r8, %r9
	negq	%r9
	xorps	%xmm1, %xmm1
	xorl	%edx, %edx
	movabsq	$13337482064, %rax      ## imm = 0x31AF9D350
	movsd	(%rax), %xmm0
	xorl	%esi, %esi
L49:	cmpq	%r8, %rdx
	jae	L121
Source line: 4
	leaq	(,%rsi,8), %rcx
Source line: 3
	movq	(%rdi), %rax
Source line: 4
	subq	%rcx, %rax
	movaps	%xmm0, %xmm2
	divsd	(%rax), %xmm2
	incq	%rdx
	decq	%rsi
	cmpq	%rsi, %r9
	addsd	%xmm2, %xmm1
	jne	L49
L98:	movabsq	$13337482064, %rax      ## imm = 0x31AF9D350
	movsd	(%rax), %xmm0
Source line: 6
	divsd	%xmm1, %xmm0
	movq	%rbp, %rsp
	popq	%rbp
	ret
L121:	movl	$1, %eax
Source line: 3
	subq	%rsi, %rax
	movq	%rsp, %rcx
	leaq	-16(%rcx), %rsi
	movq	%rsi, %rsp
	movq	%rax, -16(%rcx)
	movabsq	$jl_bounds_error_ints, %rax
	movl	$1, %edx
	callq	*%rax


## Other metaprogramming tools

`eval` evaluates an `Expr` object:

In [5]:
x = 1
eval(:(sin(x)))

0.8414709848078965

Useful for code generation: e.g. when generating wrappers around C library functions for different types (e.g. see [BLAS](https://github.com/JuliaLang/julia/blob/master/base/linalg/blas.jl) or [LAPACK](https://github.com/JuliaLang/julia/blob/master/base/linalg/lapack.jl) code).

> **Warning**: if you start writing macros, at some point you will be annoyed that macros don't have access to type information. You may be tempted to use `eval` inside a macro to get around this: *don't* do it, this can cause all sorts of problems (e.g. if you use the macro inside a function).

One way around this is using *staged functions*: these allow on-demand code generation of functions, and have access to input types.